In [1]:
import re
import time
from pathlib import Path
import shutil
import glob
import string
import unicodedata
from datetime import datetime, timedelta

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from sqlalchemy import create_engine
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from unicodedata import normalize, category

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stopset = set(stopwords.words("english"))
print(stopset)

{'not', "won't", 'further', 'wouldn', 'yourselves', 'theirs', 'where', 'how', 'for', 'should', 'in', 'my', 'them', 'which', 'or', 'so', "shan't", 'if', 'when', 'this', "mightn't", 'didn', 'ma', 'is', 'over', 'myself', 'themselves', 'at', "you've", "you're", 'him', 'y', 'aren', 'ours', 'about', 'whom', 'a', 'will', 're', 'having', 'some', 'wasn', "didn't", 'won', 'to', 'she', "wasn't", 'o', "hadn't", 'why', "hasn't", 'because', 'than', "haven't", "doesn't", 'haven', 'off', 'all', 'are', 'was', 'were', "should've", 'mightn', 'weren', 'hasn', 'it', 'don', 'through', 'herself', 'i', 'up', 'hers', 'their', 'during', 'being', 's', 'ourselves', 'an', 'each', 'her', 'only', 'under', 'your', 'we', 'doing', "don't", 'did', 'below', 'now', 'ain', 't', 'me', 'the', 'its', 'couldn', 'very', 'you', 'yourself', "aren't", 'am', 'isn', 'until', 'by', 'on', "you'd", 'down', "it's", "she's", 'here', "you'll", 'doesn', 'he', 'such', "that'll", 'no', 'into', 'd', "shouldn't", 'our', 'nor', 'before', 'own',

In [3]:
hoy = datetime.today()
hoy_format = hoy.strftime('%d%m%Y') 
hoy_format

'24102021'

In [16]:
data = pd.read_csv('deceptive-opinion.csv')
print(data.shape)

data.head()

(1600, 5)


,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


## >> Limpinado caracteres extraños

In [17]:
from unicodedata import normalize, category

def clean_text_1(_text):
    """
        Ll : minusculas
        Zs : espacios
        Lu : mayusculas
        Nd : numeros
        Mn : tildes
        Ps : abre dieresis, .....
        Pe : cierra dieresis, .......
        Pi : abre parentesis, .....
        Po : cierra parentesis, punto, coma, .....        
        ...
    """
    selects_characters = ['Ll', 'Zs', 'Lu']
    
    return ''.join([_.lower() for _ in normalize('NFD', _text) if category(_) in selects_characters])

In [18]:
data['text'] = data['text'].apply(clean_text_1)
data.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,we stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,triple a rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,this comes a little late as im finally catchin...
3,truthful,omni,positive,TripAdvisor,the omni chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,i asked for a high floor away from the elevato...


### QUITANDO STOPWORDS

In [19]:
stopset_cleaned = [clean_text_1(word) for word in stopset]
print(stopset_cleaned)

['not', 'wont', 'further', 'wouldn', 'yourselves', 'theirs', 'where', 'how', 'for', 'should', 'in', 'my', 'them', 'which', 'or', 'so', 'shant', 'if', 'when', 'this', 'mightnt', 'didn', 'ma', 'is', 'over', 'myself', 'themselves', 'at', 'youve', 'youre', 'him', 'y', 'aren', 'ours', 'about', 'whom', 'a', 'will', 're', 'having', 'some', 'wasn', 'didnt', 'won', 'to', 'she', 'wasnt', 'o', 'hadnt', 'why', 'hasnt', 'because', 'than', 'havent', 'doesnt', 'haven', 'off', 'all', 'are', 'was', 'were', 'shouldve', 'mightn', 'weren', 'hasn', 'it', 'don', 'through', 'herself', 'i', 'up', 'hers', 'their', 'during', 'being', 's', 'ourselves', 'an', 'each', 'her', 'only', 'under', 'your', 'we', 'doing', 'dont', 'did', 'below', 'now', 'ain', 't', 'me', 'the', 'its', 'couldn', 'very', 'you', 'yourself', 'arent', 'am', 'isn', 'until', 'by', 'on', 'youd', 'down', 'its', 'shes', 'here', 'youll', 'doesn', 'he', 'such', 'thatll', 'no', 'into', 'd', 'shouldnt', 'our', 'nor', 'before', 'own', 'while', 'but', 'sh

In [20]:
data['text'] = data['text'].apply(lambda _: ' '.join([word for word in _.split() if word not in stopset_cleaned]))
data.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,stayed one night getaway family thursday tripl...
1,truthful,hyatt,positive,TripAdvisor,triple rate upgrade view room less also includ...
2,truthful,hyatt,positive,TripAdvisor,comes little late im finally catching reviews ...
3,truthful,omni,positive,TripAdvisor,omni chicago really delivers fronts spaciousne...
4,truthful,hyatt,positive,TripAdvisor,asked high floor away elevator got room pleasa...


### LEMMATIZACION

In [21]:
#!pip install spacy                                # -- descomentar la 1era vez

In [22]:
#!pip install stanza                               #  -- descomentar la 1era vez

In [23]:
#!pip install "spacy-stanza<0.3.0" --user          #  -- descomentar la 1era vez

In [24]:
import stanza

In [30]:
#stanza.download('en')                              #  -- descomentar la 1era vez

In [26]:
from spacy_stanza import StanzaLanguage

In [31]:
snlp = stanza.Pipeline(lang="en")
nlp = StanzaLanguage(snlp)

2021-10-24 21:24:26 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-10-24 21:24:26 INFO: Use device: cpu
2021-10-24 21:24:26 INFO: Loading: tokenize
2021-10-24 21:24:27 INFO: Loading: pos
2021-10-24 21:24:28 INFO: Loading: lemma
2021-10-24 21:24:28 INFO: Loading: depparse
2021-10-24 21:24:28 INFO: Loading: sentiment
2021-10-24 21:24:29 INFO: Loading: ner
2021-10-24 21:24:29 INFO: Done loading processors!


In [38]:
data['lemmas'] = data['text'].apply(lambda _: ' '.join([token.lemma_ for token in nlp(_)]))
data.head()

,deceptive,hotel,polarity,source,text,lemmas
0,truthful,conrad,positive,TripAdvisor,stayed one night getaway family thursday tripl...,stay one night getaway family Thursday triple ...
1,truthful,hyatt,positive,TripAdvisor,triple rate upgrade view room less also includ...,triple rate upgrade view room less also includ...
2,truthful,hyatt,positive,TripAdvisor,comes little late im finally catching reviews ...,come little late im finally catching review pa...
3,truthful,omni,positive,TripAdvisor,omni chicago really delivers fronts spaciousne...,omni chicago really deliver front spaciousness...
4,truthful,hyatt,positive,TripAdvisor,asked high floor away elevator got room pleasa...,ask high floor away elevator get room pleasant...


In [37]:
data.iloc[0]['text']

'stayed one night getaway family thursday triple aaa rate steal th floor room complete plasma tv bose stereo voss evian water gorgeous bathroomno tub fine us concierge helpful cannot beat location flaw breakfast pricey service slowhours four kids four adults friday morning even though two tables restaurant food good worth wait would return heartbeat gem chicago'

## >> DOCUMENT-TERM-MATRIX ( DTM ) basado en TDF.IDF

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

vect = TfidfVectorizer(
    stop_words=stopset_cleaned,
    max_features=1000
)

In [43]:
vect_text = vect.fit_transform(data['lemmas'])

In [44]:
vect_text

<1600x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 76356 stored elements in Compressed Sparse Row format>

### BOLSA DE PALABRAS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv_fit = cv.fit_transform(textos)

In [ ]:
cv_fit

In [ ]:
print([name for name in dir(cv_fit) if '__' not in name])

In [ ]:
print(cv.get_feature_names())

In [ ]:
print(cv.vocabulary_)

In [ ]:
tamanio_vocabulario = len(cv.vocabulary_)
tamanio_vocabulario

In [ ]:
cantidad_textos = len(textos)
cantidad_textos

In [ ]:
espacios_con_ceros = cv_fit.toarray().sum(axis=1).sum()
espacios_con_ceros

### Porcentaje de ceros

In [ ]:
sparsity = 1 - 172/ (119*14)
sparsity